### Descrição:

#### O objetivo deste exercício é treinar e avaliar um classificador Naive Bayes para uma tarefa de análise de sentimentos.

#### O texto utilizado no treinamento e no teste deve ser pré-processado para incluir o prefixo “NAO_” de acordo com a seguinte regra:

  O prefixo “NAO_” deve ser adicionado a cada palavra após um
  token de negação (não, nem, nunca, jamais, tampouco) até a próxima
  pontuação. São consideradas pontuações: . , ? ! ;
  O treino deve ser realizado com 80% dos dados e o teste com 20%. Você deve
  calcular a acurácia do classificador aplicado aos dados de teste e comparar com
  o resultado do Naive Bayes sem a utilização do prefixo “NAO_”.


In [1]:
!wget https://github.com/yurimalheiros/pln/raw/main/datasets/B2W-Reviews01.csv.zip

--2023-04-20 22:46:54--  https://github.com/yurimalheiros/pln/raw/main/datasets/B2W-Reviews01.csv.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yurimalheiros/pln/main/datasets/B2W-Reviews01.csv.zip [following]
--2023-04-20 22:46:54--  https://raw.githubusercontent.com/yurimalheiros/pln/main/datasets/B2W-Reviews01.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20530574 (20M) [application/zip]
Saving to: ‘B2W-Reviews01.csv.zip.2’

B2W-Reviews01.csv.z 100%[===================>]  19.58M   129MB/s    in 0.2s    

2023-04-20 22:46:55 (129 MB/s) - ‘B2W-Reviews01.csv.zip.2’ saved [20530

In [2]:
!unzip B2W-Reviews01.csv.zip

Archive:  B2W-Reviews01.csv.zip
replace B2W-Reviews01.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: B2W-Reviews01.csv       
  inflating: __MACOSX/._B2W-Reviews01.csv  


In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [19]:
# Nltk serve para fazer a tokenização
nltk.download('punkt')

# Lendo o arquivo csv e removendo as linhas com valores nulos
df = pd.read_csv('B2W-Reviews01.csv').dropna().reset_index().drop(columns=['index'])

# Mesma coisa para os dados que vão receber Nao_
df_nao = pd.read_csv('B2W-Reviews01.csv').dropna().reset_index().drop(columns=['index'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-19-d1139d7e7f5b>:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('B2W-Reviews01.csv').dropna().reset_index().drop(columns=['index'])
<ipython-input-19-d1139d7e7f5b>:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nao = pd.read_csv('B2W-Reviews01.csv').dropna().reset_index().drop(columns=['index'])


In [20]:
# Verificando a quantidade de cada nota
df['overall_rating'].value_counts()

# Classificação dos dados em positivo e negativo
df['label'] = np.where(df['overall_rating'] >= 3, 'pos', 'neg')

In [21]:
df['label'].value_counts()

pos    31749
neg     6917
Name: label, dtype: int64

In [22]:
# Removendo linhas com valores nulos
df.dropna(inplace=True, subset=['review_text'])

In [23]:
# Instanciando countvectorizer
vect = CountVectorizer()

In [24]:
# Separando em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df['review_text'], df['label'], test_size=0.2)

In [25]:
# fit and transform apenas no treino
X_train = vect.fit_transform(X_train)

# transform apenas no teste
X_test = vect.transform(X_test)

In [26]:
# Instanciando naive bayes   
nb = MultinomialNB()

# Treinando o modelo
nb.fit(X_train, y_train)

MultinomialNB()

In [27]:
nb.predict(X_test)

array(['pos', 'pos', 'neg', ..., 'pos', 'pos', 'pos'], dtype='<U3')

In [28]:
# Tokenização

count_vectorizer = CountVectorizer() #instancia o objeto
word_tokenizer = count_vectorizer.build_tokenizer() #cria o tokenizador
neg_list = word_tokenizer('não nem nunca jamais tampouco') #lista de palavras negativas

# Adiciona “NAO_” a cada palavra após neg_list até a próxima pontuação
def add_negation(text):

    text = text.lower() #deixa tudo em minúsculo
    tokens = nltk.word_tokenize(text) #tokeniza
    new_tokens = [] #lista de tokens
    neg = False #flag para indicar se a palavra é negativa ou não


    for token in tokens:
        if token in neg_list: #se a palavra for negativa
            neg = True #ativa a flag
       
        if token in ['.', '!', '?',',',';']: #se a palavra for pontuação
            neg = False #desativa a flag

        if neg and token not in neg_list: #se a palavra for negativa e não estiver na lista de negativas
            token = 'NAO_' + token #adiciona NAO_ na frente da palavra
        new_tokens.append(token) #adiciona a palavra na lista de tokens
          
            
    return ' '.join(new_tokens) #retorna a lista de tokens como uma string


df_nao['review_text'] = df_nao['review_text'].apply(add_negation) #aplica a função no dataset Nao_

In [29]:
# Classificação dos dados em positivo e negativo
df_nao['label'] = np.where(df_nao['overall_rating'] >= 3, 'pos', 'neg')

# Removendo linhas com valores nulos
df_nao.dropna(inplace=True, subset=['review_text'])

# Instanciando countvectorizer
vect_nao = CountVectorizer()

# Separa os dados em treino e teste
X_train_nao, X_test_nao, y_train_nao, y_test_nao = train_test_split(df_nao['review_text'], df_nao['label'], test_size=0.2)

# Fit and transform apenas no treino
X_train_nao = vect_nao.fit_transform(X_train_nao)

# Transform apenas no teste
X_test_nao = vect_nao.transform(X_test_nao)

# Instancia naive bayes
nb_nao = MultinomialNB()

# Treina o modelo
nb_nao.fit(X_train_nao, y_train_nao)

nb_nao.predict(X_test_nao)

array(['pos', 'pos', 'pos', ..., 'pos', 'pos', 'neg'], dtype='<U3')

In [30]:
# Relatório de classficação
print("Normal")
print(classification_report(y_test, nb.predict(X_test)))
print('\n')
print("\nCom NAO_")
print(classification_report(y_test_nao, nb_nao.predict(X_test_nao)))

Normal
              precision    recall  f1-score   support

         neg       0.77      0.83      0.80      1364
         pos       0.96      0.95      0.95      6370

    accuracy                           0.93      7734
   macro avg       0.87      0.89      0.88      7734
weighted avg       0.93      0.93      0.93      7734




Com NAO_
              precision    recall  f1-score   support

         neg       0.75      0.83      0.79      1355
         pos       0.96      0.94      0.95      6379

    accuracy                           0.92      7734
   macro avg       0.86      0.89      0.87      7734
weighted avg       0.93      0.92      0.92      7734



In [31]:
# Com NAO_
my_input = ['não gostei da televisão, jamais comprarei']
my_input = [add_negation(my_input[0])]

my_input_vect = vect_nao.transform(my_input)

nb_nao.predict(my_input_vect)

array(['neg'], dtype='<U3')

In [32]:
#Normal
my_input2 = ['não gostei da televisão, jamais comprarei']

my_input_vect2 = vect.transform(my_input2)

nb.predict(my_input_vect2)

array(['pos'], dtype='<U3')

### Os dados que foram tratados com o prefixo NAO_ apresentam uma acertividade maior em sentenças negativas. Isso se da pelo fato dele punir palavras com um peso muito grande, nesse caso a palavra "gostei".